In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

# 이미지 전처리를 위한 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 이미지 특성 추출을 위한 ResNet 모델 로드
resnet = models.resnet50(pretrained=True)
modules = list(resnet.children())[:-1]  # 마지막 FC 레이어 제거
resnet = nn.Sequential(*modules)
for param in resnet.parameters():
    param.requires_grad = False



# 이미지 캡션 생성을 위한 LSTM 모델 정의
class CaptioningModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(CaptioningModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, features, captions, lengths):
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs

# 하이퍼파라미터 정의
embed_size = 256
hidden_size = 512
vocab_size = len(vocab)  # 단어 사전 크기
num_layers = 1

# 모델 초기화
model = CaptioningModel(embed_size, hidden_size, vocab_size, num_layers)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 데이터셋 및 데이터로더 설정
# 데이터셋 및 데이터로더 설정은 실제 데이터셋과 연결하여 적절한 설정이 필요합니다.

# 모델 학습
num_epochs = 5
for epoch in range(num_epochs):
    for images, captions, lengths in dataloader:
        images = images.cuda()
        captions = captions.cuda()
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        
        # 이미지 특성 추출
        features = resnet(images)
        features = features.view(features.size(0), -1)
        
        # 예측
        outputs = model(features, captions, lengths)
        
        # 손실 계산 및 역전파
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 로그 출력
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
